## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Weather-Populate-Table


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- Call a HTTP endpoint and download and parse the hourly weather data data:
    1. For each city: "New York City", "London", "Tokyo", "San Francisco" read the current days events
    2. Call to Weather service
    3. Insert the data into BigQuery as JSON

- Note: The code is currently not calling a London or Japan specific weather service.  The code needs to be enhanced.
    1. If you want to code London see: https://www.metoffice.gov.uk/services/data/datapoint
    2. If you want to code Japan see: ?

## Initialize Python

In [ ]:
# No need to set these
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
nyc_url="https://api.weather.gov/gridpoints/OKX/33,35/forecast/hourly"
san_francisco_url="https://api.weather.gov/gridpoints/MTR/85,105/forecast/hourly"

# NOTE: These are not used since they are in not in the US and need to be coded
london_url=""
tokyo_url=""

# NOTE: These are used as "similar"
seattle_url="https://api.weather.gov/gridpoints/SEW/125,68/forecast/hourly"  # Like London
washington_dc="https://api.weather.gov/gridpoints/LWX/96,71/forecast/hourly" # Like Tokyo

if london_url == "":
  london_url = seattle_url

if tokyo_url == "":
  tokyo_url = washington_dc


city_urls=[nyc_url,london_url,tokyo_url,san_francisco_url]

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

## Create the GenAI Insights table

In [ ]:
%%bigquery
CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_data_beans_curated_dataset}.weather`
--CREATE OR REPLACE TABLE `${project_id}.${bigquery_data_beans_curated_dataset}.weather` -- only use to replace the whole table
(
    weather_id    INT   NOT NULL OPTIONS(description="The primary key of the weather."),
    city_id       INT   NOT NULL OPTIONS(description="The foreign key of the city."),
    weather_date  DATE  NOT NULL OPTIONS(description="The date of the weather."),
    weather_json  JSON  NOT NULL OPTIONS(description="The weather JSON payload from the REST API."),
)
CLUSTER BY weather_id;

In [ ]:
%%bigquery

-- Remove the current days events so you re-run
DELETE
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather`
 WHERE weather_date = CURRENT_DATE();

## Supporting Functions

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

## Get the Weather per City




- https://www.weather.gov/documentation/services-web-api
- NYC:  40.7143,-74.006
  1. https://api.weather.gov/points/40.7143,-74.006
  2. https://api.weather.gov/gridpoints/OKX/33,35/forecast/hourly

- San Francisco: 37.7749, -122.4194
  1. https://api.weather.gov/points/37.7749,-122.4194
  2. https://api.weather.gov/gridpoints/MTR/85,105/forecast/hourly

In [ ]:
from datetime import datetime
import requests
import json

weather_id = GetNextPrimaryKey("${project_id}.${bigquery_data_beans_curated_dataset}.weather","weather_id")
print(f"weather_id: {weather_id}")

weather_date = datetime.today().strftime('%Y-%m-%d')

# Loop for each city
for city_index in range(0, 4):
  print(f"City: {city_ids[city_index]}")
  response = requests.get(city_urls[city_index])

  if response.status_code == 200:
    # format json
    response_json = json.loads(response.text)
    weather_json = json.dumps(response_json)

    sql = f"""INSERT INTO `${project_id}.${bigquery_data_beans_curated_dataset}.weather`
                          (weather_id, city_id, weather_date, weather_json)
                  VALUES ({weather_id},{city_ids[city_index]},'{weather_date}',JSON'{weather_json}')"""

    print(f"sql: {sql}")

    RunQuery(sql)
    weather_id = weather_id + 1
  else:
    errorMessage = f"REAT API (serviceJob) response returned response.status_code: {response.status_code} for city: {city_ids[city_index]}"
    raise Exception(errorMessage)

## See the results

In [ ]:
%%bigquery

# Display the results
SELECT *
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather`
 WHERE weather_date = CURRENT_DATE()
ORDER BY weather_id

#### Sample Formatted JSON

```
{
    "@context": [
        "https://geojson.org/geojson-ld/geojson-context.jsonld",
        {
            "@version": "1.1",
            "@vocab": "https://api.weather.gov/ontology#",
            "geo": "http://www.opengis.net/ont/geosparql#",
            "unit": "http://codes.wmo.int/common/unit/",
            "wx": "https://api.weather.gov/ontology#"
        }
    ],
    "properties": {
        "elevation": {
            "unitCode": "wmoUnit:m",
            "value": 2.1336
        },
        "forecastGenerator": "HourlyForecastGenerator",
        "generatedAt": "2024-03-14T13:50:03+00:00",
        "periods": [
            {
                "detailedForecast": "",
                "dewpoint": {
                    "unitCode": "wmoUnit:degC",
                    "value": 5.555555555555555
                },
                "endTime": "2024-03-14T10:00:00-04:00",
                "icon": "https://api.weather.gov/icons/land/day/few,0?size=small",
                "isDaytime": true,
                "name": "",
                "number": 1,
                "probabilityOfPrecipitation": {
                    "unitCode": "wmoUnit:percent",
                    "value": 0
                },
                "relativeHumidity": {
                    "unitCode": "wmoUnit:percent",
                    "value": 74
                },
                "shortForecast": "Sunny",
                "startTime": "2024-03-14T09:00:00-04:00",
                "temperature": 50,
                "temperatureTrend": null,
                "temperatureUnit": "F",
                "windDirection": "W",
                "windSpeed": "5 mph"
            }
        ],
        "units": "us",
        "updateTime": "2024-03-14T11:27:56+00:00",
        "updated": "2024-03-14T11:27:56+00:00",
        "validTimes": "2024-03-14T05:00:00+00:00/P7DT20H"
    },
    "type": "Feature"
}
```